### 2. Setup

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://seller-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential
Handle Credentials
Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: offer_received
Attributes: [{'name': 'ProductionDate', 'value': '1672981261'}, {'name': 'ExpirationDate', 'value': '1704517261'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


### 3. Go to manufacturer notebook to send credential

### 8. Get Credential Record

In [4]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'credential_exchange_id': '603cf9e1-8a88-4335-9246-6e887bb66a4b', 'thread_id': '675dbd61-631e-49e1-929f-05c9daa4b111', 'trace': True, 'schema_id': 'T8ocwUDfMeVFzuVt5ExD2d:2:Devices_Schema:1.0.4', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '5b7ab7d7-1fc8-4b07-b7f2-83b0d67f9946', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'ProductionDate', 'value': '1672981261'}, {'name': 'ExpirationDate', 'value': '1704517261'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]}, 'cred_def_id': 'T8ocwUDfMeVFzuVt5ExD2d:3:CL:78:default', 'comment': 'create automated credential exchange', 'schema_id': 'T8ocwUDfMeVFzuVt5ExD2d:2:Devices_Schema:1.0.4'}, 'created_at': '2023-01-06 05:01:04.913587Z', 'auto_offer

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: offer_received
Being offered: [{'name': 'ProductionDate', 'value': '1672981261'}, {'name': 'ExpirationDate', 'value': '1704517261'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


### 9. Send Request for Record

In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: request_sent
Handle Credentials
Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: request_sent
Attributes: [{'name': 'ProductionDate', 'value': '1672981261'}, {'name': 'ExpirationDate', 'value': '1704517261'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]
Handle Credentials
Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: credential_received
Attributes: [{'name': 'ProductionDate', 'value': '1672981261'}, {'name': 'ExpirationDate', 'value': '1704517261'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


### 10. Get Record

In [7]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: credential_received


### 11. Seller store the credential in his wallet

In [8]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My Device Credential_0")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: credential_acked
Attributes: [{'name': 'ProductionDate', 'value': '1672981261'}, {'name': 'ExpirationDate', 'value': '1704517261'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]
Credential exchange 603cf9e1-8a88-4335-9246-6e887bb66a4b, role: holder, state: credential_acked


## **Terminate**

In [9]:
response = await agent_controller.terminate()
